# Create necessary classes

## General classes

In [ ]:
# package(s) related to the simulation
import simpy

# package(s) related to time, space and id
import datetime
import json
import geojson
import shapely.geometry
import uuid

# package(s) for data handling
import numpy as np

In [ ]:
class Identifiable(object):
    """Something that has a name and id
    
    env: a simpy Environment
    name: a name
    id: a unique id generated with uuid"""
    
    def __init__(self, env, name, id=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.env = env
        self.name = name
        # generate some id, in this case based on m
        self.id = id if id else str(uuid.uuid1())
                

In [ ]:
class Location(object):
    """Something with a geometry (geojson format)
    
    geometry: can be a point as well as a polygon"""
    
    def __init__(self, geometry, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.geometry = geometry


In [ ]:
class Container(object):
    """Container class
    
    capacity: amount the container can hold
    level: amount the container holds
    container: a simpy object that can hold stuff
    total_requested: a counter needed to prevent over-handling"""

    def __init__(self, capacity, level=0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.container = simpy.Container(env, capacity, init=level)
        self.total_requested = 0


In [ ]:
class Move(object):
    """Move class
    
    origin: origin of trip
    destination: destination of trip
    v_empty: speed empty [m/s]
    v_full: speed full [m/s]
    resource: a simpy resource that can be requested"""
    
    def __init__(self, 
                 origin, destination,
                 v_empty=1.5, v_full=1, 
                 nr_resources=1,
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.origin = origin
        self.destination = destination
        self.v_empty = v_empty 
        self.v_full = v_full 
        self.resource = simpy.Resource(env, capacity=nr_resources)

    def execute_move(self, origin, destination):
        """determine distance between origin and destination, and
        yield the time it takes to travel it"""
        origin = shapely.geometry.asShape(self.origin.geometry)
        destination = shapely.geometry.asShape(self.destination.geometry)
        distance = origin.distance(destination) 

        if self.container.level == self.container.capacity:
            yield self.env.timeout(distance / self.v_full)
            print('  sailing full: ' + str(self.v_full) + ' m/s')
 
        elif self.container.level == 0:
            yield self.env.timeout(distance / self.v_empty)
            print('  sailing empty: ' + str(self.v_empty) + ' m/s')
 

In [ ]:
class Process(object):
    """Process class
    
    resource: a simpy resource that can be requested
    origin: object with simpy Container from which to get (can be Site or Vessel)
    destination: object with simpy Container in which to put (can be Site or Vessel)
    rate: rate with which quantity can be processed [amount/s]
    amount: amount to process"""

    def __init__(self, 
                 nr_resources,
                 origin, destination, rate, amount=0,
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.resource = simpy.Resource(env, capacity=nr_resources)
        self.origin=origin
        self.destination=destination
        self.rate=rate
        self.amount=amount
        
    def execute_process(self, origin, destination, amount):
        """get amount from origin container,
        put amount in destination continater, and
        yield the time it takes to process it"""
        origin.container.get(amount)
        destination.container.put(amount)
        yield self.env.timeout(amount / self.rate)
        

In [ ]:
class Log(object):
    """Log class
    
    log: log message [format: 'start activity' or 'stop activity']
    t: timestamp
    value: a value can be logged as well"""
        
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.log = []
        self.t = []
        self.value = []
        
    def log_entry(self, log, t, value):
        """Log"""
        self.log.append(log)
        self.t.append(t)
        self.value.append(value)
        

## Constructed classes

In [ ]:
class Site(Identifiable, Location, Log, Container):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [ ]:
class Transport_Resource(Identifiable, Location, Log, Container, Move):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [ ]:
class Transport_Processing_Resource(Identifiable, Location, Log, Container, Move, Process):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [ ]:
class Processing_Resource(Identifiable, Location, Log, Process):
    def __init__(self, 
                 *args, **kwargs):
        super().__init__(*args, **kwargs)


## Activities

In [ ]:
class Installation(Identifiable):
    """The Installation Class forms a spefic class of activities with associated methods that can 
    initiate and suspend processes according to a number of specified conditions. 
    
    This class deals with transport and installation/placement of discrete and continuous objects.
    
    condition: expression that states when to initiate or to suspend activity
    origin: object with simpy Container from which to get (can be Site or Vessel)
    destination: object with simpy Container in which to put (can be Site or Vessel)
    loader: gets amount from origin Contaner and puts it into mover Container
    mover: moves amount in Container from origin to destination
    unloader: gets amount from mover Contaner and puts it into destination Container"""

    def __init__(self, 
                 condition,
                 origin, destination,  
                 loader, mover, unloader,
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        
        self.condition = condition
        
        self.origin = origin
        self.destination = destination
        
        self.loader = loader
        self.mover = mover
        self.unloader = unloader
        
        self.standing_by_proc = env.process(
            self.standing_by(env, 
                             condition,
                             origin, destination,
                             loader, mover, unloader))

        self.installation_proc = env.process(
            self.installation_process_control(env,
                             condition,
                             origin, destination,
                             loader, mover, unloader))

        self.installation_reactivate = env.event()

    def standing_by(self, env, condition,
                          origin, destination,
                          loader, mover, unloader):
        """Standing by"""
        shown = False

        while not eval(condition):
            if not shown:
                print('T=' + '{:06.2f}'.format(env.now) + ' ' + self.name + ' to ' + destination.name + ' suspended')
                shown = True
            yield env.timeout(3600) # step 1 time unit ahead

        print('T=' + '{:06.2f}'.format(env.now) + ' ' + 'Condition: ' + condition + ' is satisfied')

        self.installation_reactivate.succeed()  # "reactivate"
        self.installation_reactivate = env.event()
 
    def installation_process_control(self, env, condition,
                                           origin, destination,
                                           loader, mover, unloader):
        """Installation process control"""  
        while not eval(condition):
            yield self.installation_reactivate

        print('T=' + '{:06.2f}'.format(env.now) + ' '+ self.name + ' to ' + destination.name + ' started')
        while eval(condition):
            yield from self.installation_process(env, condition,
                                                 origin, destination,
                                                 loader, mover, unloader)
        
    def installation_process(self, env, condition,
                                   origin, destination,
                                   loader, mover, unloader):
        """Installation process"""
        amount = min(
            mover.container.capacity - mover.container.level,
            destination.container.capacity - destination.container.level)

        if amount>0:
            # request access to the load_resource
            with loader.resource.request() as my_load_resource_turn:
                yield my_load_resource_turn

                destination.total_requested += amount
                origin.total_requested += amount

                mover.log_entry('loading start', self.env.now, mover.container.level)
                print('Loading:')
                yield from loader.execute_process(origin, mover, amount)
                mover.log_entry('loading stop', self.env.now, mover.container.level)
 
                loader.resource.release(my_load_resource_turn)

                print('  ' + origin.name + ' contains: ' + str(origin.container.level))
                print('  ' + mover.name + ' contains: ' + str(mover.container.level))
                print('  ' + destination.name + ' contains: ' + str(destination.container.level))
 
            # request access to the transport_resource
            with mover.resource.request() as my_mover_turn:
                yield my_mover_turn

                mover.log_entry('sailing full start', self.env.now, mover.container.level)
                yield from mover.execute_move(origin, destination)
                mover.log_entry('sailing full stop', self.env.now, mover.container.level)

                # request access to the placement_resource
                with unloader.resource.request() as my_unloader_turn:
                    yield my_unloader_turn

                    mover.log_entry('unloading start', self.env.now, mover.container.level)
                    print('Unloading:')
                    yield from loader.execute_process(mover, destination, amount)
                    mover.log_entry('unloading stop', self.env.now, mover.container.level)

                    unloader.resource.release(my_unloader_turn)

                    print('  ' + origin.name + ' contains: ' + str(origin.container.level))
                    print('  ' + mover.name + ' contains: ' + str(mover.container.level))
                    print('  ' + destination.name + ' contains: ' + str(destination.container.level))

#            # request access to the transport_resource
#            with mover.resource.request() as my_mover_turn:
#                yield my_mover_turn

                mover.log_entry('sailing full start', self.env.now, mover.container.level)
                yield from mover.execute_move(destination, origin)
                mover.log_entry('sailing full stop', self.env.now, mover.container.level)

                mover.resource.release(my_mover_turn)


# Start case

In [ ]:
import simpy

In [ ]:
# *** Create a project environment
env = simpy.Environment()

In [ ]:
Sites = []
# *** Generate stock sites
# - sites in database
data_stock_01 = {"env": env,
                "name": "Stock 01", "geometry": geojson.Point([5.019298185633251, 52.94239823421129]),
                "capacity": 150000, "level": 150000}
data_stock_02 = {"env": env,
                "name": "Stock 02", "geometry": geojson.Point([5.019298185633251, 52.94239823421129]),
                "capacity": 150000, "level": 150000}
data_stock_03 = {"env": env,
                "name": "Stock 03", "geometry": geojson.Point([5.019298185633251, 52.94239823421129]),
                "capacity": 150000, "level": 150000}
data_stock_04 = {"env": env,
                "name": "Stock 04", "geometry": geojson.Point([5.019298185633251, 52.94239823421129]),
                "capacity": 150000, "level": 150000}

# - create site objects
stock_01 = Site(**data_stock_01)
Sites.append(stock_01)
stock_02 = Site(**data_stock_02)
Sites.append(stock_02)
stock_03 = Site(**data_stock_03)
Sites.append(stock_03)
stock_04 = Site(**data_stock_04)
Sites.append(stock_04)

# *** Generate placemnet sites
start = [5.051984474450297,52.9389114955695] # Den Oever 
#stop = [5.201697426528369,53.01679552959467] # Breezanddijk 
stop = [5.294441318495398,53.06556661924486] # Kornwerderzand
nums = 20

# - generate a 'nums' amount of sites between the selected start and stop points
lats = np.linspace(start[0], stop[0], num=nums)
lons = np.linspace(start[1], stop[1],  num=nums)

# - option to create a range of sites between two points
for i in range(nums):
    # - sites in database (nr indicates km's from Den Oever haven)
    data_site = {"env": env,
                "name": "KP" + format(i,'02.0f'), "geometry": geojson.Point([lats[i], lons[i]]),
                "capacity": 5000, "level": 0}
    
    # - create site objects
    vars()['Site_' + "KP" + format(i,'02.0f')] = Site(**data_site)
    Sites.append(vars()['Site_' + "KP" + format(i,'02.0f')])


In [ ]:
# *** Define fleet

# sites in database (nr indicates km's from Den Oever haven)
# - processing resources
data_gantry_crane = {"env": env,
                "name": "Gantry crane", "geometry": geojson.Point([52.94239823421129, 5.019298185633251]),
                "origin": stock_01, "destination": stock_01, "rate": 0.10, "nr_resources": 1}
data_installation_crane = {"env": env,
                "name": "Installation crane", "geometry": geojson.Point([53.0229621352376,  5.197016484858931]),
                "origin": stock_01, "destination": stock_01, "rate": 0.05, "nr_resources": 1}

# - transport resources
data_transport_barge_01 = {"env": env,
                "name": "Transport barge 01", "geometry": geojson.Point([52.93917167503315, 5.070195628786471]),
                "capacity": 1000, "level": 0, "nr_resources": 1, "origin": stock_01, "destination": stock_01}
data_transport_barge_02 = {"env": env,
                "name": "Transport barge 02", "geometry": geojson.Point([52.93917167503315, 5.070195628786471]),
                "capacity": 1000, "level": 0, "nr_resources": 1, "origin": stock_01, "destination": stock_01}

# create site objects
# - processing resources
gantry_crane = Processing_Resource(**data_gantry_crane)
installation_crane = Processing_Resource(**data_installation_crane)

# - transport resources
transport_barge_01 = Transport_Resource(**data_transport_barge_01)
transport_barge_02 = Transport_Resource(**data_transport_barge_02)

# print the outputs
print(gantry_crane.__dict__)
print(transport_barge_01.__dict__)
print(transport_barge_02.__dict__)
print(installation_crane.__dict__)


In [ ]:
# *** Define activties

for i in range(nums):
    # - sites in database (nr i indicates km's from Den Oever haven)
    if i==0:
        condition = "'''Site_KP" + format(i,'02.0f') + ".container.level<5000'''"
        data_act = {"env": env,
                "name": "Block placement",
                "origin": stock_01, "destination": vars()['Site_' + "KP" + format(i,'02.0f')],
                "loader": gantry_crane, "mover": transport_barge_01, "unloader": installation_crane,
                "condition": eval(condition)}
    else:
        condition = "'''" + eval("'''Site_KP" + format(i,'02.0f') + ".container.level<5000'''") + \
                    ' and ' + eval("'''Site_KP" + format(i-1,'02.0f') + ".container.level>=3000'''") + "'''"
        data_act = {"env": env,
                "name": "Block placement",
                "origin": stock_01, "destination": vars()['Site_' + "KP" + format(i,'02.0f')],
                "loader": gantry_crane, "mover": transport_barge_01, "unloader": installation_crane,
                "condition": eval(condition)}
        
    # - create site objects
    vars()['Act_' + format(i,'02.0f')] = Installation(**data_act)
    

In [ ]:
#*** Run the project
env.run()

# Some basic visualisation on Google Earth

In [ ]:
from simplekml import Kml, Style

kml = Kml()

fol = kml.newfolder(name="A Folder")

sharedstyle = Style()
sharedstyle.labelstyle.color = 'ffffffff'  # White
sharedstyle.labelstyle.scale = 1  
sharedstyle.iconstyle.color = 'ffff0000'  # Blue
sharedstyle.iconstyle.scale = 1  
sharedstyle.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/shapes/placemark_circle.png'

for site in Sites:
    pnt = fol.newpoint(name=site.name, coords=[site.geometry["coordinates"]])
    pnt.timespan.begin = "2011-02-20"
    pnt.timespan.end = "2012-07-31"
    pnt.style = sharedstyle                

kml.save("sharedstyle.kml")